In [8]:
import json

import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr_detect(image_path: str, appkey: str):
    """
    detect api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/detect'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data})


def kakao_ocr_recognize(image_path: str, boxes: list, appkey: str):
    """
    recognize api request example
    :param boxes: 감지된 영역 리스트. Canvas 좌표계: 좌상단이 (0,0) / 우상단이 (limit,0)
                    감지된 영역중 좌상단 점을 기준으로 시계방향 순서, 좌상->우상->우하->좌하
                    ex) [[[0,0],[1,0],[1,1],[0,1]], [[1,1],[2,1],[2,2],[1,2]], ...]
    :param image_path: 이미지 파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/recognize'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data}, data={"boxes": json.dumps(boxes)})


def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    #image_path, appkey = sys.argv[1], sys.argv[2]
    image_path, appkey = "3.jpg", "0f7afc62141be4a96389d07160d79381"
    
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr_detect(image_path, appkey).json()
    print("[detect] output:\n{}\n".format(output))

    boxes = output["result"]["boxes"]
    boxes = boxes[:min(len(boxes), LIMIT_BOX)]
    output = kakao_ocr_recognize(image_path, boxes, appkey).json()
    
    result = json.dumps(output, sort_keys=True, indent=2, ensure_ascii=False)
    print("[recognize] output:\n{}\n".format(result))

if __name__ == "__main__":
    main()

[detect] output:
{'result': {'boxes': [[[254, 25], [338, 25], [338, 45], [254, 45]], [[112, 155], [164, 157], [163, 175], [111, 173]], [[113, 176], [155, 176], [155, 197], [113, 197]], [[202, 186], [223, 186], [223, 201], [202, 201]], [[106, 223], [157, 223], [157, 243], [106, 243]], [[282, 225], [312, 225], [312, 244], [282, 244]], [[343, 225], [374, 225], [374, 244], [343, 244]], [[419, 225], [460, 225], [460, 246], [419, 246]], [[118, 267], [210, 267], [210, 285], [118, 285]], [[332, 269], [380, 269], [380, 327], [332, 327]], [[408, 269], [466, 269], [466, 327], [408, 327]], [[119, 288], [165, 288], [165, 303], [119, 303]], [[120, 307], [220, 309], [220, 328], [119, 326]], [[121, 332], [168, 332], [168, 346], [121, 346]], [[338, 332], [380, 332], [380, 346], [338, 346]], [[416, 332], [465, 332], [465, 348], [416, 348]], [[119, 371], [168, 371], [168, 387], [119, 387]], [[119, 353], [168, 353], [168, 367], [119, 367]], [[162, 390], [226, 390], [226, 406], [162, 406]], [[179, 369], [2